##Para criar um exemplo de um sistema RAG (Retrieval-Augmented Generation) que funcione com PDFs em português, precisaremos realizar as seguintes etapas:

Extrair o texto dos PDFs.
Codificar o texto extraído.
Criar um índice de pesquisa de similaridade.
Recuperar documentos relevantes com base em uma consulta.
Gerar uma resposta com base nos documentos recuperados.


Primeiro, vamos instalar as bibliotecas necessárias:

In [ ]:
!pip install PyPDF2 faiss-cpu transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.8 MB/s eta 0:00:00


##Passo 1: Extrair Texto dos PDFs
Usaremos a biblioteca PyPDF2 para extrair texto dos PDFs.

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text


##Passo 2: Codificar o Texto Extraído
Vamos usar um transformador de sentenças pré-treinado para codificar o texto extraído.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def encode(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

##Passo 3: Criar um Índice FAISS
Vamos criar um índice FAISS para busca rápida de similaridade.

In [ ]:
import faiss
import numpy as np

corpus = [extract_text_from_pdf("documento1.pdf"), extract_text_from_pdf("documento2.pdf")]  # Adicione os caminhos dos seus PDFs
corpus_embeddings = encode(corpus)

d = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(corpus_embeddings.numpy())


##Passo 4: Recuperar Documentos Relevantes
Vamos recuperar os documentos mais relevantes do corpus com base em uma consulta.

In [ ]:
def retrieve(query, top_k=3):
    query_embedding = encode([query]).numpy()
    distances, indices = index.search(query_embedding, top_k)
    return [corpus[i] for i in indices[0]]

query = "que é um neurônios?"
relevant_docs = retrieve(query)
print(relevant_docs)

['66 \nRede Neurais Artificiais - RNAs  \n4  \nRedes Neurais Artificiais – RNAs \nRedes neurais artificial (RNA) são algoritmos que s e baseiam no comportamento \ndo cérebro humano. Dessa forma, imita a estrutura m assivamente paralela do cérebro, \ncom capacidade de adquirir, armazenar e utilizar co nhecimentos experimentais \n(HAYKIN, 2001). Apresentam como característica a ro bustez e tolerância à falhas, ou \nseja, mesmo diante do mau funcionamento de determin ada neurônio a rede neural se \nmantém estável, produzindo resultados confiáveis. \nSeu funcionamento é bastante semelhante ao sistema biológico em dois aspectos: \no conhecimento é obtido através de processos de apr endizado e a densidade das sinapses \n(conexões entre neurônios) é a chave para se armaze nar os conhecimentos adquiridos. \nA estrutura das RNAs é formada por diversas unidade s de processamento, \nchamadas neurônios artificiais, que se encontram in terligadas. Essa rede de neurônios \nartificiais se comunica at

##Passo 5: Gerar uma Resposta
Vamos usar um modelo generativo para gerar uma resposta com base nos documentos recuperados.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

def generate_answer(retrieved_docs, query):
    context = " ".join(retrieved_docs)
    input_text = f"Contexto: {context} Pergunta: {query} Resposta:"
    inputs = gpt_tokenizer(input_text, return_tensors="pt", truncation=True, max_length=300) # Increased max_length for input
    outputs = gpt_model.generate(**inputs, max_new_tokens=100)  # Generate up to 100 new tokens
    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

answer = generate_answer(relevant_docs, query)
print(answer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Contexto: 66 
Rede Neurais Artificiais - RNAs  
4  
Redes Neurais Artificiais – RNAs 
Redes neurais artificial (RNA) são algoritmos que s e baseiam no comportamento 
do cérebro humano. Dessa forma, imita a estrutura m assivamente paralela do cérebro, 
com capacidade de adquirir, armazenar e utilizar co nhecimentos experimentais 
(HAYKIN, 2001). Apresentam como característica a ro bustez e tolerância à falhas, ou 
seja, mesmo diante do mau funcionamento de determin ada neurônio a rede neural se 
mantém estável, produzindo resultados confiáveis. 
Seu funcionamento é bastante semelhante ao sistema biológico em dois aspectos: 
o conhecimento é obtido através de processos de apr endizado e a densidade das sinapses 
(conexões entre neurônios) é a chave para se armaze nar conseguir,

sejando a la même de la même de la même de la même de la même de la même de la même de la même de la même de la même de la même de la même de la même de la même de la même de la même de la même de la même
